In [2]:
!pip3 install -U sagemaker

In [3]:
import os
import boto3
import sagemaker
import pandas as pd
#import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = "internet-churn-linear-regression"
s3_private_path_tsv = "s3://{}/final/tsv".format(bucket)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
df = pd.read_csv("../clean_churn.csv")
print(df.columns)

df = df.drop(labels= ['Unnamed: 0', 'id'], axis=1)
df = df[df[df.columns].min(axis=1) >= 0]

#df = df.rename(columns = {'reamining_contract':'remaining_contract'})
#df['remaining_contract'] = df['remaining_contract'].astype(str)

#df['remaining_contract'].replace('nan', 'no contract', inplace=True)

df[['download_avg','upload_avg']] = df[['download_avg','upload_avg']].fillna(df[['download_avg','upload_avg']].median())
        
#df = pd.get_dummies(df, columns = ['remaining_contract'],dtype = int)
# df= df.rename({'remaining_contract_0-1 years':'remaining_contract_0-1_years',
#               'remaining_contract_1-2 years': 'remaining_contract_1-2_years',
#               'remaining_contract_2-3 years': 'remaining_contract_2-3_years',
#               'remaining_contract_no contract':'remaining_contract_no_contract'},axis = 1)


df.head(5)

Index(['Unnamed: 0', 'id', 'is_tv_subscriber', 'is_movie_package_subscriber',
       'subscription_age', 'bill_avg', 'service_failure_count', 'download_avg',
       'upload_avg', 'download_over_limit', 'churn',
       'remaining_contract_0-1_years', 'remaining_contract_1-2_years',
       'remaining_contract_2-3_years', 'remaining_contract_no_contract'],
      dtype='object')


,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,service_failure_count,download_avg,upload_avg,download_over_limit,churn,remaining_contract_0-1_years,remaining_contract_1-2_years,remaining_contract_2-3_years,remaining_contract_no_contract
0,1,0,11.95,25,0,8.4,2.3,0,0,1,0,0,0
1,0,0,8.22,0,0,0.0,0.0,0,1,0,0,0,1
2,1,0,8.91,16,0,13.7,0.9,0,1,1,0,0,0
3,0,0,6.87,21,1,0.0,0.0,0,1,0,0,0,1
4,0,0,6.39,0,0,0.0,0.0,0,1,0,0,0,1


In [5]:
# Split into train and test, validation 80/10/10
X = df.drop(labels = 'churn', axis = 1)
y = df['churn']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state = 24)
X_test,X_val,y_test,y_val = train_test_split(X_test,y_test,test_size = 0.5,random_state = 24)

In [6]:
# Scale variables
cols_scale = ['subscription_age','bill_avg','service_failure_count','download_avg','upload_avg','download_over_limit']
SS = ColumnTransformer([('scaler',StandardScaler(),cols_scale)],remainder='passthrough')
X_train = SS.fit_transform(X_train)
X_test = SS.fit_transform(X_test)
X_val = SS.fit_transform(X_val)

In [7]:
new_cols = ['subscription_age','bill_avg','service_failure_count',
                                           'download_avg','upload_avg','download_over_limit','is_tv_subscriber',
                                           'is_movie_package_subscriber','remaining_contract_0-1_years',
                                           'remaining_contract_1-2_years','remaining_contract_2-3_years',
                                           'remaining_contract_no_contract']
X_train = pd.DataFrame(X_train,columns = new_cols)
X_test = pd.DataFrame(X_test,columns = new_cols)
X_val = pd.DataFrame(X_val,columns = new_cols)
X_train.head()

,subscription_age,bill_avg,service_failure_count,download_avg,upload_avg,download_over_limit,is_tv_subscriber,is_movie_package_subscriber,remaining_contract_0-1_years,remaining_contract_1-2_years,remaining_contract_2-3_years,remaining_contract_no_contract
0,2.502725,0.232133,-0.335453,-0.121422,-0.190893,-0.209741,1.0,0.0,0.0,1.0,0.0,0.0
1,0.772996,0.005874,-0.335453,0.974460,0.390738,-0.209741,1.0,0.0,0.0,1.0,0.0,0.0
2,1.282604,0.307553,-0.335453,-0.487231,-0.354158,-0.209741,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.226620,0.382972,-0.335453,0.363236,0.472371,-0.209741,0.0,0.0,0.0,0.0,0.0,1.0
4,0.135986,-0.069546,-0.335453,-0.450187,-0.323546,-0.209741,1.0,0.0,0.0,0.0,0.0,1.0


In [8]:
# Append y cols to dataframes
y_train.index = list(range(0,len(y_train)))
train_df = X_train
train_df["churn"] = y_train
col_names = list(train_df.columns)
col_names = [col_names[-1]] + col_names[:-1]
train_df = train_df[col_names]
train_df["churn"] = train_df["churn"].astype(int)

y_val.index = list(range(0,len(y_val)))
val_df = X_val
val_df["churn"] = y_val
col_names = list(val_df.columns)
col_names = [col_names[-1]] + col_names[:-1]
val_df = val_df[col_names]
val_df["churn"] = val_df["churn"].astype(int)

# y_test.index = list(range(0,len(y_test)))
test_df = X_test
# test_df["churn"] = y_test
# col_names = list(test_df.columns)
# col_names = [col_names[-1]] + col_names[:-1]
# test_df = test_df[col_names]
# test_df["churn"] = test_df["churn"].astype(int)

In [9]:
# Write split dfs to csv files
train_df.to_csv("../data/splits/train.csv", header=False, index=False)
val_df.to_csv("../data/splits/val.csv", header=False, index=False)
test_df.to_csv("../data/splits/test.csv", header=False, index=False)

In [10]:
!aws s3 cp --recursive ../data/splits/ $s3_private_path_tsv/

upload: ../data/splits/test.csv to s3://sagemaker-us-east-1-035349649198/final/tsv/test.csv
upload: ../data/splits/val.csv to s3://sagemaker-us-east-1-035349649198/final/tsv/val.csv
upload: ../data/splits/train.csv to s3://sagemaker-us-east-1-035349649198/final/tsv/train.csv


In [11]:
!aws s3 ls $s3_private_path_tsv/

2024-02-13 02:27:04    1049475 test.csv
2024-02-13 02:27:04    8347937 train.csv
2024-02-13 02:27:04    1059318 val.csv


In [13]:
%%time
from time import gmtime, strftime
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
import boto3

# Set up job name and output location
job_name = "linear-learner-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "{}/linear_learner_output/{}".format(s3_private_path_tsv, job_name)

# Retrieve the image URI for linear learner
region_name = boto3.Session().region_name
linear_learner_image = sagemaker.image_uris.retrieve(
    framework="linear-learner", 
    region=region_name
)

# Set up the estimator
ll_estimator = sagemaker.estimator.Estimator(
    linear_learner_image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=50,
    output_path=output_location,
    sagemaker_session=sess,
)

# Set hyperparameters for logistic regression
ll_estimator.set_hyperparameters(
    predictor_type="binary_classifier",
    mini_batch_size=100,
)

# Data channels
train_data = sagemaker.inputs.TrainingInput(
    "{}/train.csv".format(s3_private_path_tsv),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    "{}/val.csv".format(s3_private_path_tsv),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data, "validation": validation_data}

# Start training
ll_estimator.fit(inputs=data_channels, job_name=job_name, logs=True)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: linear-learner-2024-02-13-02-32-08


2024-02-13 02:32:08 Starting - Starting the training job...
2024-02-13 02:32:22 Starting - Preparing the instances for training......
2024-02-13 02:33:38 Downloading - Downloading input data...
2024-02-13 02:34:08 Downloading - Downloading the training image......
2024-02-13 02:34:58 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[02/13/2024 02:35:10 INFO 140121997047616] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss'

In [14]:
!aws sagemaker list-training-jobs --sort-by CreationTime --sort-order Descending

{
    "TrainingJobSummaries": [
        {
            "TrainingJobName": "linear-learner-2024-02-13-02-32-08",
            "TrainingJobArn": "arn:aws:sagemaker:us-east-1:035349649198:training-job/linear-learner-2024-02-13-02-32-08",
            "CreationTime": 1707791528.512,
            "TrainingEndTime": 1707791965.599,
            "LastModifiedTime": 1707791966.006,
            "TrainingJobStatus": "Completed"
        },
        {
            "TrainingJobName": "linear-learner-2024-02-13-02-30-57",
            "TrainingJobArn": "arn:aws:sagemaker:us-east-1:035349649198:training-job/linear-learner-2024-02-13-02-30-57",
            "CreationTime": 1707791457.448,
            "TrainingEndTime": 1707791884.059,
            "LastModifiedTime": 1707791884.525,
            "TrainingJobStatus": "Completed"
        },
        {
            "TrainingJobName": "linear-learner-2024-02-10-07-46-07",
            "TrainingJobArn": "arn:aws:sagemaker:us-east-1:035349649198:training-job/linear-learn

In [15]:
!aws s3 ls $s3_private_path_tsv/lr_output/linear-learner-2024-02-13-02-32-08/linear-learner-2024-02-13-02-32-08/output/

In [16]:
sagemaker_client = boto3.client('sagemaker')
training_job_name = 'linear-learner-2024-02-13-02-32-08'

response = sagemaker_client.describe_training_job(TrainingJobName=training_job_name)

metrics = response.get('FinalMetricDataList', [])

# Print each metric
for metric in metrics:
    print(f"{metric['MetricName']}: {metric['Value']}")

train:progress: 100.0
train:objective_loss: 0.4032808542251587
validation:binary_f_beta: 0.8402524590492249
validation:objective_loss: 0.40841802954673767
validation:objective_loss:final: 0.4085102081298828
validation:binary_classification_accuracy: 0.817930281162262
validation:roc_auc_score: 0.8990910053253174
validation:recall: 0.856047511100769
validation:precision: 0.8250297904014587
train:throughput: 3797.8857421875


In [18]:
ll_transformer = ll_estimator.transformer(1, "ml.m4.xlarge")

# start a transform job
input_location = "{}/test.csv".format(s3_private_path_tsv)  # use input data without ID column
ll_transformer.transform(input_location, content_type="text/csv", split_type="Line")
ll_transformer.wait()

INFO:sagemaker:Creating model with name: linear-learner-2024-02-13-02-42-09-485
INFO:sagemaker:Creating transform job with name: linear-learner-2024-02-13-02-42-10-233


.............................................
Docker entrypoint called with argument(s): serve
Docker entrypoint called with argument(s): serve
Running default environment configuration script
[02/13/2024 02:49:37 INFO 140199140177728] Memory profiler is not enabled by the environment variable ENABLE_PROFILER.
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
/opt/amazon/lib/python3.8/site-packages/scipy/optimize/_shgo.py:495: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if cons['type'] is 'ineq':
/opt/amazon/lib/python3.8/site-packages/scipy/optimize/_shgo.py:743: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(self.X_min) is not 0:
Running default environment configuration script
[02/13/2024 02:49:37 INFO 140199140177728] Memory profiler is not enabled by the environment variable ENABLE_PROFILER.
/opt/amazon/lib/python3.8/site-packages/mxnet/

In [73]:
import re

def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

In [74]:
s3 = boto3.client("s3")
output_df = get_csv_output_from_s3(ll_transformer.output_path, "test.csv")
output_df.head(8)

,0,1
0,"{""predicted_label"":0",score:0.505380630493164}
1,"{""predicted_label"":0",score:0.040701072663068}
2,"{""predicted_label"":1",score:0.935273468494415}
3,"{""predicted_label"":1",score:0.942014694213867}
4,"{""predicted_label"":1",score:0.929572880268096}
5,"{""predicted_label"":0",score:0.292035341262817}
6,"{""predicted_label"":0",score:0.108596071600914}
7,"{""predicted_label"":1",score:0.538442730903625}


In [75]:
output_df = output_df.drop([0], axis= 1)

In [76]:
output_df.rename(columns={1: "pred"}, inplace=True)

In [77]:
output_df.head()

,pred
0,score:0.505380630493164}
1,score:0.040701072663068}
2,score:0.935273468494415}
3,score:0.942014694213867}
4,score:0.929572880268096}


In [78]:
output_df['pred'] = output_df['pred'].str.replace('score:', '').str.replace('}', '').astype(float)

In [79]:
output_df.head()

,pred
0,0.505381
1,0.040701
2,0.935273
3,0.942015
4,0.929573


In [80]:
output_df.columns = ["pred"]
output_df['pred'] = output_df['pred'].apply(lambda x: 0 if x < 0.5 else 1)
output_df

,pred
0,1
1,0
2,1
3,1
4,1
...,...
7222,0
7223,0
7224,1
7225,1


In [81]:
from sklearn.metrics import classification_report
classification_report(y_test, output_df["pred"], output_dict=True)

{'0': {'precision': 0.8289874703691161,
  'recall': 0.7569573283858998,
  'f1-score': 0.7913366736705996,
  'support': 3234.0},
 '1': {'precision': 0.8160973327094057,
  'recall': 0.8735286751815677,
  'f1-score': 0.8438369420587879,
  'support': 3993.0},
 'accuracy': 0.8213643282136432,
 'macro avg': {'precision': 0.8225424015392608,
  'recall': 0.8152430017837338,
  'f1-score': 0.8175868078646937,
  'support': 7227.0},
 'weighted avg': {'precision': 0.8218655221644359,
  'recall': 0.8213643282136432,
  'f1-score': 0.8203436712732058,
  'support': 7227.0}}